# Generate simulated data to link

In this case study, we imagine running a person linkage on a simulated 2030 Census Unedited File (CUF) --
see the main notebook for more details, including references used throughout this notebook.
This notebook creates simulated input (CUF) and reference files approximating what would be used in such a process,
emulating the methods found in publicly available descriptions of the Census Bureau's primary person linkage system, PVS.

In [1]:
# Query planning is now on by default, but it has some rough edges.
# See https://github.com/dask/dask/issues/10995 for general discussion
# and https://github.com/dask/dask-expr/issues/1060 for the particular
# issue I ran into.
import dask
dask.config.set({"dataframe.query-planning": False})

In [2]:
import os, time
import warnings
import datetime
# Importing pandas for access, regardless of whether we are using it as the compute engine
import pandas
import numpy
from IPython.display import display

In [3]:
%load_ext autoreload
%autoreload 1

In [4]:
%aimport vivarium_research_prl
from vivarium_research_prl import distributed_compute, utils

In [5]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
# DO NOT EDIT if this notebook is not called generate_simulated_data.ipynb!
# This notebook is designed to be run with papermill; this cell is tagged 'parameters'
data_to_use = 'small_sample'
output_dir = '../output/generate_simulated_data'
compute_engine = 'pandas'
num_jobs = 10
cpus_per_job = 2
threads_per_job = 1
memory_per_job = "10GB"
local_directory = f"/tmp/{os.environ['USER']}_{int(time.time())}_dask"

In [7]:
# Parameters
data_to_use = "usa"
output_dir = "/mnt/team/simulation_science/priv/users/zmbc/prl/person_linkage_case_study/generate_simulated_data/"
compute_engine = "dask"
num_jobs = 50
memory_per_job = "120GB"
cpus_per_job = 5
local_directory = "/mnt/team/simulation_science/priv/users/zmbc/prl/person_linkage_case_study_tmp/dask/"


In [8]:
if compute_engine.startswith('dask'):
    utils.ensure_empty(local_directory)

In [9]:
print(datetime.datetime.now())

2024-05-22 11:18:18.969676


In [10]:
df_ops, pd = distributed_compute.start_compute_engine(
    compute_engine,
    num_jobs=num_jobs,
    cpus_per_job=cpus_per_job,
    threads_per_job=threads_per_job,
    memory_per_job=memory_per_job,
    local_directory=local_directory,
)

<Client: 'tcp://10.158.111.40:43839' processes=50 threads=50, memory=5.46 TiB>

In [11]:
def save_file_with_ground_truth(file_name, file, ground_truth):
    # Check that file and ground truth have the same records
    assert (
        # record_id is unique
        len(file) ==
        len(df_ops.drop_duplicates(file[['record_id']]))
    )
    assert (
        len(ground_truth) ==
        len(file[['record_id']].merge(ground_truth[['record_id']], on='record_id', how='inner'))
    )

    file_path = f'{output_dir}/{data_to_use}/{file_name}.parquet'
    utils.remove_path(file_path)
    file.to_parquet(file_path)

    ground_truth_path = f'{output_dir}/{data_to_use}/{file_name}_ground_truth.parquet'
    utils.remove_path(ground_truth_path)
    ground_truth.to_parquet(ground_truth_path)

## Load pseudopeople simulated datasets

### Record ID tracking (data lineage)

We do a little bit of work here to enable tracking the "ground truth" (the simulant IDs from
pseudopeople).
We give each pseudopeople record/row a unique identifier for tracking, and then we immediately
separate the ground truth information (which we would not have if we were using real data)
from the rest of the columns (which we would have).
The ground truth is only used in the specific "ground truth" section of this notebook,
to help avoid accidentally leaking information into the case study.

Since we also combine/aggregate pseudopeople records as part of the process of generating the
simulated reference files, ground truth is a bit more complicated than you might imagine.
For example, the ground truth may tell us that a single row in a reference file is actually
a composite of several individuals, because even the deterministic linkage (by SSN) we use
here is not without error.

We handle this by tracking *all* source records used in the construction of each record in
our reference files.
This is achieved by having a table mapping composite record IDs to the "source record IDs"
(IDs of records that were directly generated by pseudopeople).
When we aggregate records, this is combined accordingly.

In [12]:
def add_unique_id_col(df, col_name='unique_id', value_prefix=''):
    if compute_engine == 'pandas' or compute_engine.startswith('modin'):
        return df.reset_index().rename(columns={'index': col_name}).assign(**{col_name: lambda df: value_prefix + df[col_name].astype(str)})
    elif compute_engine.startswith('dask'):
        # Can use cumsum as in https://stackoverflow.com/a/60852409/ if it needs
        # to be incrementing, but we just need uniqueness

        def add_id_to_partition(df_part, partition_info=None):
            return (
                df_part
                    .assign(**{col_name: range(len(df_part))})
                    .assign(**{col_name: lambda x: (
                            value_prefix +
                            str(partition_info['number'] if partition_info is not None else 0) +
                            '_' +
                            x[col_name].astype(str)
                        ).astype('large_string[pyarrow]')}
                    )
            )

        df = df.map_partitions(add_id_to_partition)

        return df_ops.ensure_large_string_capacity(df)
    else:
        raise ValueError()

def add_unique_record_id(df, dataset_name):
    return add_unique_id_col(df, col_name='record_id', value_prefix=f'{dataset_name}_')

# Initializes a table listing the pairs between record_ids and source record_ids.
# Should only be called on "source records"; that is, records that
# come directly out of pseudopeople.
def record_id_to_single_source_record_pairs(df, source_col='record_id'):
    if source_col == 'record_id':
        # We can't have duplicate column names, so we make a new column
        # literally called 'source_col'
        df = df.assign(source_col=lambda df: df[source_col])
        source_col = 'source_col'

    return df[['record_id', source_col]].rename(columns={source_col: 'source_record_id'})

In [13]:
# Operations that aggregate records, combining the source record pairs
# between all records that are aggregated

def merge_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix, *args, **kwargs):
    assert len(dfs) == len(source_record_pairings)
    for df in dfs:
        assert 'record_id' in df.columns

    on = kwargs.get('on', None)
    if on is not None:
        # If there are nulls in any of the merge columns, they can't match to anything
        dfs = [df.dropna(subset=on, how='any') for df in dfs]

    result = dfs[0]
    source_record_pairs = source_record_pairings[0]
    dfs_and_source_record_pairs_to_combine = list(zip(dfs[1:], source_record_pairings[1:]))
    for index, (df_to_merge, source_record_pairs_to_merge) in enumerate(dfs_and_source_record_pairs_to_combine):
        result = (
            result.merge(df_to_merge, *args, **kwargs)
        )
        if index == len(dfs_and_source_record_pairs_to_combine) - 1:
            # Since this is the last step, these are the record_ids that will actually be returned
            accumulate_step_record_id_prefix = new_record_id_prefix
        else:
            # A dummy intermediate -- this shouldn't be exposed to the user
            accumulate_step_record_id_prefix = f'merge_iter_{index}'

        result = df_ops.rebalance(add_unique_record_id(result, accumulate_step_record_id_prefix))
        source_record_pairs = df_ops.concat([
            # The pairs that were already in result
            source_record_pairs
                .rename(columns={'record_id': 'record_id_x'})
                .merge(result[['record_id', 'record_id_x']].dropna(how='any'), on='record_id_x')
                .drop(columns=['record_id_x']),
            # The new ones
            source_record_pairs_to_merge
                .rename(columns={'record_id': 'record_id_y'})
                .merge(result[['record_id', 'record_id_y']].dropna(how='any'), on='record_id_y')
                .drop(columns=['record_id_y']),
        ])
        result = result.drop(columns=['record_id_x', 'record_id_y'])

    return result, source_record_pairs


def dedupe_preserving_source_records(df, source_record_pairs, columns_to_dedupe, new_record_id_prefix):
    result = df_ops.drop_duplicates(df[columns_to_dedupe])
    result = add_unique_record_id(result, new_record_id_prefix)
    df_to_result_mapping = (
        df[['record_id'] + columns_to_dedupe]
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(result, on=columns_to_dedupe)
            [['record_id', 'record_id_pre_dedupe']]
    )
    result_source_record_pairs = (
        source_record_pairs
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(df_to_result_mapping, on='record_id_pre_dedupe')
            .drop(columns=['record_id_pre_dedupe'])
    )
    return result, result_source_record_pairs


def concat_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix):
    dfs = [df.rename(columns={'record_id': 'record_id_pre_concat'}) for df in dfs]
    result = df_ops.concat(dfs, ignore_index=True)
    result = add_unique_record_id(result, new_record_id_prefix)

    record_id_mapping = (
        result[['record_id', 'record_id_pre_concat']]
    )
    validate_kwarg = {
        'validate': 'm:1',
    } if not compute_engine.startswith('dask') else {} # Dask doesn't support validate

    all_source_record_pairings = df_ops.persist(df_ops.concat(source_record_pairings, ignore_index=False))
    result_source_record_pairs = df_ops.persist(
        all_source_record_pairings
            .rename(columns={'record_id': 'record_id_pre_concat'})
            .merge(record_id_mapping, on='record_id_pre_concat', **validate_kwarg)
            .drop(columns=['record_id_pre_concat'])
    )

    if compute_engine.startswith('dask'):
        # Manual alternative to the 'validate' kwarg
        assert len(result_source_record_pairs) == len(all_source_record_pairings)

    return result.drop(columns=['record_id_pre_concat']), result_source_record_pairs

### Custom noise types

A few noise types are really important here, but not implemented in pseudopeople, so we implement them as "add-ons."

In [14]:
def split_columns_incorrectly(df: pandas.DataFrame, columns: list[str], row_probability: float, seed: int = 1234):
    rng = numpy.random.default_rng(seed)

    to_split_incorrectly = pandas.Series(rng.random(len(df)) < row_probability, index=df.index)
    full_name = df.loc[to_split_incorrectly, columns].fillna('').agg(' '.join, axis=1).str.strip()

    num_spaces_or_hyphens = full_name.str.count('[ -]')
    split_name = full_name.str.split('[ -]', regex=True, expand=True).fillna('').values

    split_points = numpy.ceil(
        numpy.sort(rng.random((len(full_name), len(columns) - 1)), axis=1) * num_spaces_or_hyphens.values.reshape((len(full_name), 1))
    ).astype(int)
    string_start_points_1d = numpy.arange(split_name.shape[0], dtype=int) * split_name.shape[1]
    string_end_points_1d = numpy.append(string_start_points_1d[1:], [split_name.shape[0] * split_name.shape[1]])
    last_split_points_1d = None

    for split_index, column_name in enumerate(columns):
        # Map into 1D space
        new_column = split_name.copy().reshape(-1)

        if last_split_points_1d is not None:
            erase_before_slices = tuple(
                slice(string_start_point, last_split_point)
                for string_start_point, last_split_point in zip(string_start_points_1d, last_split_points_1d)
            )
        else:
            erase_before_slices = tuple()

        if split_index < split_points.shape[1]:
            split_points_1d = numpy.arange(split_name.shape[0], dtype=int) * split_name.shape[1] + split_points[:, split_index]
            erase_after_slices = tuple(
                slice(split_point, string_end_point)
                for split_point, string_end_point in zip(split_points_1d, string_end_points_1d)
            )
        else:
            erase_after_slices = tuple()

        new_column[numpy.r_[erase_before_slices + erase_after_slices]] = ''
        new_column[new_column != ''] += ' '
        df.loc[to_split_incorrectly, column_name] = (new_column.reshape(split_name.shape)).sum(axis=1)
        df.loc[to_split_incorrectly, column_name] = df.loc[to_split_incorrectly, column_name].str.strip()

        last_split_points_1d = split_points_1d

    return df

In [15]:
def shuffle_columns(df: pandas.DataFrame, columns: list[str], row_probability: float, seed: int = 1234):
    rng = numpy.random.default_rng(seed)

    to_shuffle = pandas.Series(rng.random(len(df)) < row_probability, index=df.index)
    original_values = df.loc[to_shuffle, columns].values
    random_ordering = numpy.argsort(rng.random(original_values.shape), axis=1)
    df.loc[to_shuffle, columns] = numpy.take_along_axis(original_values, random_ordering, axis=1)
    
    return df

In [16]:
def truncate_column(df: pandas.DataFrame, column: str, cell_probability: float, truncation_length: int, seed: int = 1234):
    rng = numpy.random.default_rng(seed)
    to_truncate = pandas.Series(rng.random(len(df)) < cell_probability, index=df.index)
    df.loc[to_truncate, 'last_name'] = df.loc[to_truncate, 'last_name'].str[:truncation_length]
    return df

In [17]:
def apply_custom_noise_type(df, noise_type, *args, seed: int = 1234, **kwargs):
    if compute_engine.startswith('dask'):
        return df.map_partitions(lambda partition, partition_info=None: noise_type(
            partition,
            *args,
            seed=(seed + (partition_info['number'] if partition_info is not None else 1) * 10_000),
            **kwargs,
        ), meta=df)
    else:
        return noise_type(
            df,
            *args,
            seed=seed,
            **kwargs,
        )

### 1040 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files -- see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [18]:
tax_years = list(range(2025, 2030))
tax_years

[2025, 2026, 2027, 2028, 2029]

In [19]:
%%time

# Combine 1040 for all years.
simulated_taxes_1040 = df_ops.concat([
    df_ops.read_parquet(
        f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_taxes_1040_{year}.parquet',
        columns=[
            'simulant_id',
            'ssn',
            'first_name',
            'middle_initial',
            'last_name',
            'mailing_address_street_number',
            'mailing_address_street_name',
            'mailing_address_unit_number',
            'mailing_address_po_box',
            'mailing_address_city',
            'mailing_address_state',
            'mailing_address_zipcode',
        ],
    )
    for year in tax_years
], ignore_index=True)

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    1.670000e+03
mean     1.184161e+08
std      3.224761e+06
min      1.122587e+08
25%      1.161062e+08
50%      1.184967e+08
75%      1.208592e+08
max      1.235329e+08
dtype: float64
Creating partitions of 3,955MB


CPU times: user 17.2 s, sys: 1.8 s, total: 19 s
Wall time: 1min 22s


In [20]:
simulated_taxes_1040 = apply_custom_noise_type(
    simulated_taxes_1040,
    split_columns_incorrectly,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=1,
)

In [21]:
# "... many of the [IRS] records contain only the first four letters of the last name."
# (Brown et al. 2023, p.30, footnote 19)
# Note that this truncation only matters for ITIN PIKing since for SSNs that are present in SSA we use name from SSA.
PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION = 0.4 # is this a good guess at "many" in the quote above?
simulated_taxes_1040 = apply_custom_noise_type(
    simulated_taxes_1040,
    truncate_column,
    'last_name',
    cell_probability=PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION,
    truncation_length=4,
    seed=2,
)

In [22]:
simulated_taxes_1040 = apply_custom_noise_type(
    simulated_taxes_1040,
    shuffle_columns,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=3,
)

In [23]:
simulated_taxes_1040 = df_ops.persist(add_unique_record_id(simulated_taxes_1040, 'simulated_1040'))
simulated_taxes_1040_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_taxes_1040))

In [24]:
simulated_taxes_1040_ground_truth = simulated_taxes_1040[['record_id', 'simulant_id']]
simulated_taxes_1040 = simulated_taxes_1040.drop(columns=['simulant_id'])
simulated_taxes_1040, simulated_taxes_1040_ground_truth = df_ops.persist(simulated_taxes_1040, simulated_taxes_1040_ground_truth)
simulated_taxes_1040

,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,record_id
npartitions=51,,,,,,,,,,,,
,string,string,string,string,string,string,string,string,string,category[unknown],string,string
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


### W2/1099 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files --
see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [25]:
%%time

# Combine W2/1099 for all years.
simulated_w2_1099 = df_ops.concat([
    df_ops.read_parquet(
        f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_taxes_w2_and_1099_{year}.parquet',
        columns=[
            'simulant_id',
            'ssn',
            'first_name',
            'middle_initial',
            'last_name',
            'mailing_address_street_number',
            'mailing_address_street_name',
            'mailing_address_unit_number',
            'mailing_address_po_box',
            'mailing_address_city',
            'mailing_address_state',
            'mailing_address_zipcode',
        ],
    )
    for year in tax_years
], ignore_index=True)

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    1.670000e+03
mean     1.615143e+08
std      1.038348e+06
min      1.579631e+08
25%      1.607056e+08
50%      1.615846e+08
75%      1.623738e+08
max      1.638732e+08
dtype: float64
Creating partitions of 5,395MB


CPU times: user 47.8 s, sys: 6.48 s, total: 54.3 s
Wall time: 7min 2s


In [26]:
simulated_w2_1099 = apply_custom_noise_type(
    simulated_w2_1099,
    split_columns_incorrectly,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=4,
)

In [27]:
simulated_w2_1099 = apply_custom_noise_type(
    simulated_w2_1099,
    truncate_column,
    'last_name',
    cell_probability=PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION,
    truncation_length=4,
    seed=5,
)

In [28]:
simulated_w2_1099 = apply_custom_noise_type(
    simulated_w2_1099,
    shuffle_columns,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=6
)

In [29]:
simulated_w2_1099 = df_ops.persist(add_unique_record_id(simulated_w2_1099, 'simulated_w2_1099'))
simulated_w2_1099_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_w2_1099))

In [30]:
simulated_w2_1099_ground_truth = simulated_w2_1099[['record_id', 'simulant_id']]
simulated_w2_1099 = simulated_w2_1099.drop(columns=['simulant_id'])
simulated_w2_1099, simulated_w2_1099_ground_truth = df_ops.persist(simulated_w2_1099, simulated_w2_1099_ground_truth)
simulated_w2_1099

,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,record_id
npartitions=51,,,,,,,,,,,,
,string,string,string,string,string,string,string,string,string,category[unknown],string,string
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


In [31]:
simulated_taxes, simulated_taxes_source_record_pairs = concat_preserving_source_records(
    [simulated_taxes_1040, simulated_w2_1099],
    [simulated_taxes_1040_source_record_pairs, simulated_w2_1099_source_record_pairs],
    new_record_id_prefix='simulated_taxes',
)

In [32]:
simulated_taxes_1040_with_itins = simulated_taxes_1040[simulated_taxes_1040.ssn.notnull() & simulated_taxes_1040.ssn.str.startswith('9')]
simulated_taxes_1040_with_itins_source_record_pairs = simulated_taxes_1040_source_record_pairs.merge(simulated_taxes_1040_with_itins[['record_id']], on='record_id', how='inner')
simulated_taxes_1040_with_itins, simulated_taxes_1040_with_itins_source_record_pairs = df_ops.persist(simulated_taxes_1040_with_itins, simulated_taxes_1040_with_itins_source_record_pairs)

%xdel simulated_taxes_1040
%xdel simulated_taxes_1040_source_record_pairs

In [33]:
%xdel simulated_w2_1099
%xdel simulated_w2_1099_source_record_pairs

In [34]:
simulated_taxes, simulated_taxes_source_record_pairs = df_ops.persist(simulated_taxes, simulated_taxes_source_record_pairs)

### 2030 Census Unedited File (CUF)

For now, we gloss over the data schema for addresses.
We don't know how addresses would be formatted in the CUF (and it's hard to guess, because
address is not part of the Census form), but it likely would have some of these fields
(street number, street name, etc) combined.

While PVS input files do not in general have names split into first, middle, and last,
I am guessing the CUF **would** have first name, middle initial, last name (which is how pseudopeople
generates it), because that [matches the Census questionnaire](https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/questionnaires-and-instructions/questionnaires/2020-informational-questionnaire-english_DI-Q1.pdf).

In [35]:
%%time

simulated_census_2030 = df_ops.read_parquet(f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_census_2030.parquet')

CPU times: user 24.4 s, sys: 5.36 s, total: 29.8 s
Wall time: 5min 10s


In [36]:
simulated_census_2030 = apply_custom_noise_type(
    simulated_census_2030,
    split_columns_incorrectly,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=7,
)

In [37]:
simulated_census_2030 = apply_custom_noise_type(
    simulated_census_2030,
    shuffle_columns,
    ['first_name', 'middle_initial', 'last_name'],
    row_probability=0.01,
    seed=8,
)

In [38]:
simulated_census_2030 = df_ops.persist(add_unique_record_id(simulated_census_2030, 'simulated_census_2030'))

In [39]:
simulated_census_2030_ground_truth = simulated_census_2030[['record_id', 'simulant_id']]
simulated_census_2030 = simulated_census_2030.drop(columns=['simulant_id'])
simulated_census_2030, simulated_census_2030_ground_truth = df_ops.persist(simulated_census_2030, simulated_census_2030_ground_truth)
simulated_census_2030

,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year,record_id
npartitions=334,,,,,,,,,,,,,,,,,,
,string,string,string,string,string,string,string,string,string,string,category[unknown],string,string,category[unknown],category[unknown],category[unknown],int64,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [40]:
save_file_with_ground_truth('simulated_census_2030', simulated_census_2030, simulated_census_2030_ground_truth)

In [41]:
%xdel simulated_census_2030

### Simulated SSA Numident

In [42]:
%%time

simulated_ssa_numident = df_ops.read_parquet(f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_ssa_numident.parquet')

CPU times: user 3.15 s, sys: 369 ms, total: 3.52 s
Wall time: 18.3 s


In [43]:
simulated_ssa_numident = apply_custom_noise_type(
    simulated_ssa_numident,
    split_columns_incorrectly,
    ['first_name', 'middle_name', 'last_name'],
    row_probability=0.01,
    seed=9,
)

In [44]:
simulated_ssa_numident = apply_custom_noise_type(
    simulated_ssa_numident,
    shuffle_columns,
    ['first_name', 'middle_name', 'last_name'],
    row_probability=0.01,
    seed=10,
)

In [45]:
if data_to_use == 'ri':
    # Remove simulants who were never observed in Rhode Island -- this is unrealistic to real life!
    # We only do this to create a medium-sized set of reference files for computational testing.
    simulated_ssa_numident = (
        simulated_ssa_numident
            .merge(
                df_ops.drop_duplicates(simulated_taxes_1040_ground_truth[['simulant_id']]).assign(observed_in_taxes_1040=1),
                on='simulant_id',
                how='left',
            )
            .merge(
                df_ops.drop_duplicates(simulated_w2_1099_ground_truth[['simulant_id']]).assign(observed_in_taxes_w2_1099=1),
                on='simulant_id',
                how='left',
            )
            .merge(
                df_ops.drop_duplicates(simulated_census_2030_ground_truth[['simulant_id']]).assign(observed_in_census_2030=1),
                on='simulant_id',
                how='left',
            )
            .assign(observed_ever=lambda df: (df.observed_in_taxes_1040 == 1) | (df.observed_in_taxes_w2_1099 == 1) | (df.observed_in_census_2030 == 1))
            .pipe(lambda df: df[df.observed_ever])
            .drop(columns=["observed_in_taxes_1040", "observed_in_taxes_w2_1099", "observed_in_census_2030", "observed_ever"])
    )

In [46]:
%xdel simulated_census_2030_ground_truth

In [47]:
simulated_ssa_numident = df_ops.persist(add_unique_record_id(simulated_ssa_numident, 'simulated_ssa_numident'))
simulated_ssa_numident_source_record_pairs = df_ops.persist(record_id_to_single_source_record_pairs(simulated_ssa_numident))

In [48]:
simulated_ssa_numident_ground_truth = simulated_ssa_numident[['record_id', 'simulant_id']]
simulated_ssa_numident = simulated_ssa_numident.drop(columns=['simulant_id'])
simulated_ssa_numident, simulated_ssa_numident_ground_truth = df_ops.persist(simulated_ssa_numident, simulated_ssa_numident_ground_truth)
simulated_ssa_numident

,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,record_id
npartitions=334,,,,,,,,,
,string,string,string,string,string,category[unknown],category[unknown],string,string
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [49]:
source_record_ground_truth = df_ops.persist(df_ops.concat([
    simulated_ssa_numident_ground_truth,
    simulated_taxes_1040_ground_truth,
    simulated_w2_1099_ground_truth,
]).rename(columns={'record_id': 'source_record_id'}))
source_record_ground_truth

,source_record_id,simulant_id
npartitions=436,,
,string,string
,...,...
...,...,...
,...,...
,...,...


In [50]:
%xdel simulated_taxes_1040_ground_truth
%xdel simulated_w2_1099_ground_truth

## Create reference files

> The Census Numident – all Social Security Administration (SSA) Numident SSN records are
  edited (collapsed) to produce a Census Numident file that contains “one best-data record” for
  each SSN. All variants of name information for each SSN are retained in the Alternate Name
  Numident file, while all variants of date of birth data are retained in the Alternate DOB
  Numident. The SSN-PIK crosswalk file is used to attach a corresponding unique PIK value for
  each SSN value in the Census Numident file.

### Census Numident

Luque and Wagner, p. 4:
  
> The SSA Numident file contains all transactions ever recorded against any single SSN - with each entry
representing an addition or change (such as name changes) to the SSN record. This file is edited to
create the **Census Numident**, which contains one record for each SSN. Each SSN record in the Census
Numident contains name, DOB, sex, race, place of birth, parents’ name, citizenship status and date of death information.

and in footnote 5:

> Name edits, DOB reconciliation, and race identifiers are some of the edits conducted to produce this Numident
file. **The resulting Numident file contains the most recent name and DOB data.**

We are missing quite a few columns, since they are missing in pseudopeople's SSA Numident: race, place of birth, parents' name,
citizenship status.
However, I'm pretty sure none of these are used in linking.

In [51]:
def fill_dates(df, fill_with):
    return (
        # Replace invalid dates with nans
        pd.to_datetime(df.event_date, format='%Y%m%d', errors='coerce')
            .fillna(pandas.to_datetime('2100-01-01' if fill_with == 'latest' else '1900-01-01'))
    )

def best_data_from_columns(df, columns, best_is_latest=True):
    # We don't want to throw out events with a missing/invalid date, so we'll fill them with the value *least* likely to be chosen
    # (earlier than all values if taking the latest, later than all values if taking the earliest).
    fill_with = 'earliest' if best_is_latest else 'latest'

    result = (
        df
            # Without mutating the existing date column, get one that is actually
            # a date type and can be used for sorting.
            # Note: we actually convert this to an integer for sorting purposes, because Modin was having trouble
            # sorting by it as an actual datetime
            .assign(event_date_for_sort=lambda df: fill_dates(df, fill_with=fill_with).astype(numpy.int64) // 10 ** 9)
            .dropna(subset=columns, how='all')
            .pipe(lambda df: df_ops.drop_duplicates(df, subset='ssn', sort_col='event_date_for_sort', keep=('last' if best_is_latest else 'first')))
            [['record_id', 'ssn'] + columns]
    )

    return result, record_id_to_single_source_record_pairs(result)

best_name, best_name_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident,
    columns=['first_name', 'middle_name', 'last_name'],
)

best_date_of_birth, best_date_of_birth_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident,
    columns=['date_of_birth'],
)

best_date_of_death, best_date_of_death_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident[simulated_ssa_numident.event_type == 'death'],
    columns=['event_date'],
)
best_date_of_death = best_date_of_death.rename(columns={'event_date': 'date_of_death'})

simulated_census_numident, simulated_census_numident_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [best_name, best_date_of_birth, best_date_of_death],
    [best_name_source_record_pairs, best_date_of_birth_source_record_pairs, best_date_of_death_source_record_pairs],
    new_record_id_prefix='simulated_census_numident',
    on=['ssn'],
    how='left',
))
simulated_census_numident

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    6.680000e+02
mean     8.295233e+07
std      1.410317e+08
min      2.510000e+02
25%      2.510000e+02
50%      1.275850e+04
75%      1.541010e+08
max      5.634462e+08
dtype: float64
Creating partitions of 1,108MB


Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    6.680000e+02
mean     8.217353e+07
std      1.418221e+08
min      2.510000e+02
25%      2.510000e+02
50%      1.166300e+04
75%      1.119393e+08
max      6.035917e+08
dtype: float64
Creating partitions of 1,098MB


Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    6.680000e+02
mean     8.298191e+06
std      1.391966e+07
min      2.510000e+02
25%      3.590000e+02
50%      1.352000e+04
75%      7.904908e+06
max      6.332514e+07
dtype: float64
Creating partitions of 111MB


,ssn,first_name,middle_name,last_name,date_of_birth,date_of_death,record_id
npartitions=59,,,,,,,
,string,string,string,string,string,string,string
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


### Alternate Name Numident

Wagner and Layne, p. 9:

>  All variants of name information for each SSN are retained in the Alternate Name
Numident file...

In [52]:
simulated_alternate_name_numident, simulated_alternate_name_numident_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_ssa_numident,
    simulated_ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_name', 'last_name'],
    new_record_id_prefix='simulated_alternate_name_numident',
))
simulated_alternate_name_numident

,ssn,first_name,middle_name,last_name,record_id
npartitions=668,,,,,
,string,string,string,string,string
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [53]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_alternate_name_numident, by='ssn', agg_func=lambda x: x.size()).describe())

count    3.834553e+08
mean     1.021149e+00
std      1.438817e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.000000e+00
dtype: float64

In [54]:
# Show alternate names for the same simulated SSN
simulated_alternate_name_numident.merge(
    df_ops.groupby_agg_small_groups(simulated_alternate_name_numident[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner'
).sort_values('ssn').tail(100)

,ssn,first_name,middle_name,last_name,record_id
24168,899-99-4975,Diane,Sandra,Griffin,simulated_alternate_name_numident_665_3479047
24169,899-99-4975,Dian,Sandra,Griffin,simulated_alternate_name_numident_665_3479048
24520,899-99-5315,Janine,Dana,Fonseca,simulated_alternate_name_numident_665_3479193
24521,899-99-5315,Janine,Dana,<NA>,simulated_alternate_name_numident_665_3479194
24336,899-99-5341,Jeffrey,Micheal,Drake,simulated_alternate_name_numident_665_3479203
...,...,...,...,...,...
24599,899-99-9640,Billy,Henry,Luhn,simulated_alternate_name_numident_667_27
24044,899-99-9673,Jessica,Jacquelyn,<NA>,simulated_alternate_name_numident_667_42
24045,899-99-9673,Jessica,Jacquelyn,Malabanan,simulated_alternate_name_numident_667_43
24218,899-99-9915,Doris,Theresa,Polite,simulated_alternate_name_numident_667_139


### Alternate DOB Numident

Wagner and Layne, p. 9:

> ... while all variants of date of birth data are retained in the Alternate DOB
Numident.

In [55]:
simulated_alternate_dob_numident, simulated_alternate_dob_numident_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_ssa_numident,
    simulated_ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'date_of_birth'],
    new_record_id_prefix='simulated_alternate_dob_numident',
))
simulated_alternate_dob_numident

,ssn,date_of_birth,record_id
npartitions=668,,,
,string,string,string
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [56]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_alternate_dob_numident, by='ssn', agg_func=lambda x: x.size()).describe())

count    3.834553e+08
mean     1.007827e+00
std      8.812281e-02
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.000000e+00
dtype: float64

In [57]:
simulated_alternate_dob_numident.merge(
    df_ops.groupby_agg_small_groups(simulated_alternate_dob_numident[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner'
).sort_values('ssn').tail(100)

,ssn,date_of_birth,record_id
8892,899-98-6815,19321213,simulated_alternate_dob_numident_665_3044115
8893,899-98-6815,19341203,simulated_alternate_dob_numident_665_3044116
8830,899-98-7038,19Z9|224,simulated_alternate_dob_numident_665_3044225
8831,899-98-7038,19291224,simulated_alternate_dob_numident_665_3044226
8900,899-98-7068,19491219,simulated_alternate_dob_numident_665_3044240
...,...,...,...
8969,899-99-9099,19392611,simulated_alternate_dob_numident_666_210
8900,899-99-9437,19521003,simulated_alternate_dob_numident_666_358
8901,899-99-9437,19520310,simulated_alternate_dob_numident_666_359
8992,899-99-9749,19750831,simulated_alternate_dob_numident_667_82


### Name/DOB Reference File

Wagner and Layne, p. 9:

> The Name and DOB Reference files are reformatted versions of the Census Numident
and includes **all possible combinations of alternate names and dates of birth, as well as
ITIN data**. All of the reference files contain SSN/ITIN and the corresponding PIK. When
an input record is linked to a reference file, the corresponding PIK is assigned. Table 1
presents the number of observations in each of the reference files.

A slightly confusing point: sometimes the Name and DOB reference files are described
as one and the same thing, and sometimes as separate.
I believe this is because **they differ only in how they are "cut" for the PVS process:**
the name reference file is cut by first and last initial,
while the DOB reference file is cut by month and day of birth.

This is described in Wagner and Layne, p.15:

> The [DOBSearch] module matches against a re-split
version of the Numident Name Reference file, splitting the data based on month and day
of birth.

Since we handle the logic of "cutting" in the linkage process itself, we generate
a single reference file here.

Note that unlike for addresses, and unlike for the pre-processing of *input* files
(as opposed to reference files), there is no explicit nickname processing/correction here.
I am fairly sure that is accurate to the PVS, which I believe assumes that nicknames
would not be present in SSA/tax records (or at least, that the real name would appear
at least once in these records).

In [58]:
simulated_name_dob_numident_records, simulated_name_dob_numident_records_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [simulated_alternate_name_numident, simulated_alternate_dob_numident],
    [simulated_alternate_name_numident_source_record_pairs, simulated_alternate_dob_numident_source_record_pairs],
    on=['ssn'],
    how='left',
    new_record_id_prefix='name_dob_numident_records',
))
simulated_name_dob_numident_records

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    6.680000e+02
mean     1.332053e+08
std      2.951726e+05
min      1.319390e+08
25%      1.331030e+08
50%      1.332616e+08
75%      1.333985e+08
max      1.338318e+08
dtype: float64
Creating partitions of 1,780MB


Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    1.336000e+03
mean     5.457597e+07
std      6.526173e+05
min      5.368884e+07
25%      5.392663e+07
50%      5.453158e+07
75%      5.522874e+07
max      5.547942e+07
dtype: float64
Creating partitions of 1,458MB


,ssn,first_name,middle_name,last_name,date_of_birth,record_id
npartitions=52,,,,,,
,string,string,string,string,string,string
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [59]:
# Show alternate name/DOB for the same simulated SSN
simulated_name_dob_numident_records.merge(
    df_ops.groupby_agg_small_groups(simulated_name_dob_numident_records[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner',
).sort_values('ssn').tail(100)

,ssn,first_name,middle_name,last_name,date_of_birth,record_id
295762,899-99-6217,Charlene,Victoria,Perry,19330719,name_dob_numident_records_459_591809
295763,899-99-6217,Charlene,N,Perry,19330719,name_dob_numident_records_459_591810
281190,899-99-6440,James,Harold,Yerges,19321229,name_dob_numident_records_436_591708
281191,899-99-6440,James,Female Child,Yerges,19321229,name_dob_numident_records_436_591709
367108,899-99-6477,Edward,Robert,Hollenbeck,19380215,name_dob_numident_records_568_592786
...,...,...,...,...,...,...
187755,899-99-9673,Jessica,Jacquelyn,Malabanan,19831223,name_dob_numident_records_288_592256
228070,899-99-9749,Samuel,Joel,Cedillos-Martinez,19750831,name_dob_numident_records_352_591957
228071,899-99-9749,Samuel,Joel,Cedillos-Martinez,19850918,name_dob_numident_records_352_591958
18072,899-99-9915,Doris,Theresa,Polite,19330915,name_dob_numident_records_26_590962


#### Incorporating people with ITINs

Individual Taxpayer Identification Numbers (ITINs) can be issued to people who are required to file
federal taxes but are not eligible for a Social Security Number.
The most common reason for this is being an undocumented immigrant and therefore not being authorized
to work in the United States.

People without SSNs used to be impossible to assign PIKs to.
In 2011 the NORC report stated (p. 38, footnote 19):

> NORC understands that the Census Bureau has undertaken an effort to enhance the PVS reference files with IRS
files that include Individual Taxpayer Identification Numbers (ITIN). For those people who are required to file a tax
return but do not have, and may not want an SSN—such as a non-U.S. citizen—the IRS issues the taxpayer an ITIN.
This enhancement to the PVS reference file may help to match more non-U.S citizens.

By 2014 (Wagner and Layne, p. 5):

> One of the key enhancements [made in recent years] increased the coverage of the reference files by
including records for persons with Individual Taxpayer Identification Numbers assigned
by the Internal Revenue Service (ITINs) to [along with?] the SSN-based Numident data. 

I have not found a specific description of how ITIN records are constructed in any of the
publicly-available sources.
This may be because it is straightforward, or because the tax data schema is confidential.
I assume that only IRS data is used, since no other data source that I am aware of would
report ITIN.

It is stated that the ITIN records are created directly from tax filings and not
from ITIN applications (Brown et al. p. 29, footnote 16), which is convenient
because the tax filing data is what we can simulate with
pseudopeople:

> The NUMIDENT provides the PII on the SSN-holder from the issuing agency (SSA), and that PII is used in SSN
verification. **For ITINs, the Census Bureau does not have access to the ITIN applications** to the issuing agency (IRS),
so name and DOB verification of ITINs is less reliable.

"Less reliable" is a bit confusing here, because as stated above when generating
the simulated tax data, IRS data should not contain date of birth at all.
Here, we have stayed true to this by omitting it entirely.

We assume that only 1040 filings would be used for this purpose; we wouldn't expect ITINs to
show up on employer-filed W-2/1099 forms.

In [60]:
# Analogous to the process of getting alternate names and dates of birth
# from SSA, we retain all versions of the name from taxes.
simulated_name_for_itins, simulated_name_for_itins_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_taxes_1040_with_itins.rename(columns={'middle_initial': 'middle_name'}),
    simulated_taxes_1040_with_itins_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_name', 'last_name'],
    new_record_id_prefix='simulated_name_for_itins',
))
simulated_name_for_itins

,ssn,first_name,middle_name,last_name,record_id
npartitions=102,,,,,
,string,string,string,string,string
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [61]:
df_ops.compute(df_ops.groupby_agg_small_groups(simulated_name_for_itins, by='ssn', agg_func=lambda x: x.size()).describe())

count    1.517511e+07
mean     1.691975e+00
std      6.855127e-01
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      2.000000e+00
max      7.000000e+00
dtype: float64

In [62]:
# How many records in the simulated name/DOB numident are in the ITIN range?
# With normal pseudopeople settings, this shouldn't happen, because SSN in SSA is noiseless.
# If this number is >0, we may be adding ITIN records as more alternates to existing records.
df_ops.compute(simulated_name_dob_numident_records.ssn.str.startswith('9').sum())

0

In [63]:
simulated_name_dob_reference_file, simulated_name_dob_reference_file_source_record_pairs = df_ops.persist(concat_preserving_source_records(
    [simulated_name_dob_numident_records, simulated_name_for_itins],
    [simulated_name_dob_numident_records_source_record_pairs, simulated_name_for_itins_source_record_pairs],
    new_record_id_prefix='simulated_name_dob_reference_file',
))
simulated_name_dob_reference_file

,ssn,first_name,middle_name,last_name,date_of_birth,record_id
npartitions=154,,,,,,
,string,string,string,string,string,string
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


### GeoBase Reference File

Wagner and Layne, p. 9:

> PVS creates three other sets of reference
files containing Numident data: the **GeoBase Reference File**, the Name Reference File,
and the DOB Reference file.
The GeoBase Reference File appends addresses from administrative records attached
to Numident data, including all possible combinations of alternate names and dates of
birth for SSN. Addresses from administrative records are edited and processed through
commercial software product to clean and standardize address data. ITIN data is also
incorporated into the Geobase.

Luque and Wagner, p. 5:

> Reference files contain data from the Numident file enhanced with address
data obtained from federal AR [administrative records] files.<sup>8</sup>
The reference files, thus, contain all variants of a person’s name, DOB,
and sex, as well as current and recent addresses. These reference files are
referred to as the (PVS) Geobase reference file since addresses (a geographic component)
are appended to each person record.<sup>9</sup> It is important to note that there are
multiple Geobase reference files that are created depending on the vintage of the
incoming file to be processed through PVS.

> <sup>8</sup> Namely, data from the IRS, Department of Housing and Urban Development,
several files from the Department of Health and Human Services, and Selective Service.

> <sup>9</sup> In particular, the address data is cleaned and standardized and used
to construct a variable called GEOKEY. The GEOKEY variable is constructed as a subset
of the full address, and then is appended to the Numident data to create the
PVS Geobase Reference file.

We only have IRS data to use for addresses, and specifically only W-2/1099 data,
which is a limitation of this case study.
I can't find a concrete definition of "recent" -- as noted above, we use 5 years
of IRS data.
This may be longer than the true window, but this may end up making up for
the lack of non-IRS data sources, and get us closer to a realistic number of
alternate addresses.

Also, our address data comes out of pseudopeople already parsed into address parts
like street name, etc.
For more realism, pseudopeople should output a single string that we have to (imperfectly) parse apart.

I haven't been able to find out more about what kind of "subset" the geokey is.
It is unclear to me why geokey is "interesting" since it is just derived from the
address parts.

In [64]:
address_cols = [c for c in simulated_taxes.columns if 'mailing_address' in c]

def standardize_address_part(column):
    return (
        column
            # Remove leading or trailing whitespace
            .str.strip()
            # Turn any strings of consecutive whitespace into a single space
            .str.replace('\s+', ' ', regex=True)
            # Normalize case
            .str.upper()
            # Normalize the word street as described in the example quoted above
            # In reality, there would be many rules like this
            .str.replace('\b(STREET|STR)\b', 'ST', regex=True)
            # Make sure missingness is represented consistently
            .replace('', numpy.nan)
    )

In [65]:
simulated_tax_addresses = (
    simulated_taxes
        # Can only link these to the other files if they have an SSN
        .dropna(subset=['ssn'])
        [['record_id', 'ssn'] + address_cols]
        .assign(**{c: lambda df, c=c: standardize_address_part(df[c]) for c in address_cols})
        .pipe(df_ops.ensure_large_string_capacity)
)

In [66]:
simulated_addresses_by_ssn, simulated_addresses_by_ssn_source_record_pairs = df_ops.persist(dedupe_preserving_source_records(
    simulated_tax_addresses,
    simulated_taxes_source_record_pairs,
    columns_to_dedupe=['ssn'] + address_cols,
    new_record_id_prefix='addresses_by_ssn',
))
simulated_addresses_by_ssn

,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,record_id
npartitions=204,,,,,,,,,
,string,string,string,string,string,string,string,string,string
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [67]:
num_addresses = df_ops.groupby_agg_small_groups(simulated_addresses_by_ssn, by='ssn', agg_func=lambda x: x.size()).rename('size').reset_index().sort_values('size')
num_addresses

,ssn,size
npartitions=7,,
,string,int64
,...,...
...,...,...
,...,...
,...,...


In [68]:
# Show some SSNs with a lot of address variation
df_ops.compute(pd.merge(num_addresses.tail(10), simulated_addresses_by_ssn, on='ssn', how='inner').sort_values('ssn'))

,ssn,size,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,record_id
0,003-80-3167,337,1599,GOODNOW AVEN,<NA>,<NA>,BOISE,ID,83617,addresses_by_ssn_103_2634102
1,003-80-3167,337,8548,E AVENU Q12,<NA>,<NA>,SAN JOSE,CA,92025,addresses_by_ssn_103_2634103
2,003-80-3167,337,5921,MARTIN,<NA>,<NA>,KOLUMBYA,MD,21919,addresses_by_ssn_103_2634104
3,003-80-3167,337,1899,GOODNOW AVEN,<NA>,<NA>,BOISE,ID,83617,addresses_by_ssn_103_2634105
4,003-80-3167,337,1484,US 224,<NA>,<NA>,WASHINGTON,DC,20024,addresses_by_ssn_103_2634106
...,...,...,...,...,...,...,...,...,...,...
332,721-68-3624,337,10,130TH AVNUE,<NA>,<NA>,CLAVERACK,NY,11710,addresses_by_ssn_182_14113455
333,721-68-3624,337,4320,MCCLELLAND ROAD,<NA>,<NA>,WESTBOROUGH,MA,01562,addresses_by_ssn_182_14113456
334,721-68-3624,337,229,AVENUE DE DIEGO,<NA>,<NA>,SAN PEDRO,CA,91202,addresses_by_ssn_182_14113457
335,721-68-3624,337,1301,CARSWELL STREET,<NA>,<NA>,SCHNECKSVILLE,PA,16428,addresses_by_ssn_182_14113458


In [69]:
# Rough estimate of how many rows we should have in our reference file, once we do this Cartesian product
df_ops.compute(
    len(simulated_name_dob_reference_file) *
    df_ops.groupby_agg_small_groups(simulated_addresses_by_ssn, by='ssn', agg_func=lambda x: x.size()).mean()
)

858991088.5656093

In [70]:
simulated_geobase_reference_file, simulated_geobase_reference_file_source_record_pairs = df_ops.persist(merge_preserving_source_records(
    [simulated_name_dob_reference_file, simulated_addresses_by_ssn],
    [simulated_name_dob_reference_file_source_record_pairs, simulated_addresses_by_ssn_source_record_pairs],
    on=['ssn'],
    how='left',
    new_record_id_prefix='simulated_geobase_reference_file',
))
simulated_geobase_reference_file

,ssn,first_name,middle_name,last_name,date_of_birth,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,record_id
npartitions=204,,,,,,,,,,,,,
,string,string,string,string,string,string,string,string,string,string,string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [71]:
# Actual number of rows
len(simulated_geobase_reference_file)

868984409

## Track ground truth for reference files

In [72]:
def get_simulants_of_source_records(source_record_pairs, filter_record_ids=None):
    if filter_record_ids is not None:
        source_record_pairs = source_record_pairs.pipe(filter_record_ids)

    result = (
        df_ops.drop_duplicates(source_record_pairs)
            .merge(source_record_ground_truth, on='source_record_id')
            [['record_id', 'simulant_id']]
            .pipe(df_ops.drop_duplicates)
    )
    result = result.merge(
        df_ops.groupby_agg_small_groups(result, by='record_id', agg_func=lambda x: x.simulant_id.nunique().rename('n_unique_simulants')).reset_index(),
        on='record_id',
        how='left',
    )
    return result

def get_ground_truth_for_records(source_record_pairs):
    result = df_ops.persist(get_simulants_of_source_records(source_record_pairs))

    result_nunique_describe = df_ops.compute(result['n_unique_simulants'].describe())

    if numpy.isclose(result_nunique_describe.loc['max'], 1):
        print('No collisions')
        return result

    print('Collisions:')
    display(result_nunique_describe)

    print('Simulated tax records with the most collisions:')
    most_collisions_record_id = result.sort_values('n_unique_simulants', ascending=False).head(1).record_id.iloc[0]
    most_collisions_source_record_ids = df_ops.compute(
        df_ops.drop_duplicates(source_record_pairs[source_record_pairs.record_id == most_collisions_record_id][['source_record_id']])
    )
    tax_record_ids = (
        most_collisions_source_record_ids
            .merge(
                df_ops.compute(df_ops.drop_duplicates(
                    simulated_taxes_source_record_pairs[
                        simulated_taxes_source_record_pairs.source_record_id.isin(most_collisions_source_record_ids.source_record_id)
                    ][['record_id', 'source_record_id']]
                )),
                on='source_record_id',
                how='inner',
            )
    )
    most_collisions_ground_truth = df_ops.compute(
        source_record_ground_truth[source_record_ground_truth.source_record_id.isin(most_collisions_source_record_ids.source_record_id)]
    )

    most_collisions_tax_filings = (
        tax_record_ids
            .merge(
                df_ops.compute(
                    simulated_taxes[simulated_taxes.record_id.isin(tax_record_ids.record_id)],
                ),
                on='record_id',
                how='left',
            )
            .merge(
                most_collisions_ground_truth,
                on='source_record_id',
                how='left',
            )
    )
    display(most_collisions_tax_filings)

    print('Simulated SSA records with the most collisions:')
    display(
        df_ops.compute(
            most_collisions_source_record_ids
                .merge(
                    df_ops.compute(
                        simulated_ssa_numident[simulated_ssa_numident.record_id.isin(most_collisions_source_record_ids.source_record_id)]
                            .rename(columns={'record_id': 'source_record_id'})
                    ),
                    on='source_record_id',
                    how='inner'
                )
                .merge(most_collisions_ground_truth, on='source_record_id', how='left')
        )
    )

    return result

### Simulated Census Numident

In [73]:
len(simulated_census_numident_source_record_pairs)

801720506

In [74]:
simulated_census_numident_ground_truth = get_ground_truth_for_records(simulated_census_numident_source_record_pairs)

No collisions


In [75]:
simulated_census_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
npartitions=872,,,
,string,string,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


### Simulated Alternate Name Numident

In [76]:
len(simulated_alternate_name_numident_source_record_pairs)

419397698

In [77]:
simulated_alternate_name_numident_ground_truth = get_ground_truth_for_records(simulated_alternate_name_numident_source_record_pairs)

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    1.336000e+03
mean     2.927265e+07
std      5.065151e+07
min      0.000000e+00
25%      0.000000e+00
50%      9.700000e+03
75%      2.077832e+07
max      2.387286e+08
dtype: float64
Creating partitions of 782MB


No collisions


In [78]:
simulated_alternate_name_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
npartitions=872,,,
,string,string,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


### Alternate DOB Numident

In [79]:
len(simulated_alternate_dob_numident_source_record_pairs)

419397698

In [80]:
simulated_alternate_dob_numident_ground_truth = get_ground_truth_for_records(simulated_alternate_dob_numident_source_record_pairs)

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    1.336000e+03
mean     2.895475e+07
std      5.005966e+07
min      0.000000e+00
25%      0.000000e+00
50%      1.032250e+04
75%      1.985346e+07
max      2.215955e+08
dtype: float64
Creating partitions of 774MB


No collisions


In [81]:
simulated_alternate_dob_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
npartitions=872,,,
,string,string,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


### Simulated Name/DOB Reference File

In [82]:
len(simulated_name_dob_reference_file_source_record_pairs)

904661822

In [83]:
simulated_name_dob_reference_file_ground_truth = get_ground_truth_for_records(simulated_name_dob_reference_file_source_record_pairs)

No collisions


In [84]:
simulated_name_dob_reference_file_ground_truth

,record_id,simulant_id,n_unique_simulants
npartitions=872,,,
,string,string,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


### Simulated GeoBase Reference File

In [85]:
len(simulated_geobase_reference_file_source_record_pairs)

4730055815

In [86]:
simulated_geobase_reference_file_ground_truth = get_ground_truth_for_records(simulated_geobase_reference_file_source_record_pairs)

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    1.632000e+03
mean     4.004342e+07
std      7.138149e+07
min      0.000000e+00
25%      0.000000e+00
50%      2.149000e+03
75%      1.479276e+07
max      3.835086e+08
dtype: float64
Creating partitions of 1,307MB


Collisions:


count    9.318796e+08
mean     1.139533e+00
std      3.668062e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      9.000000e+00
Name: n_unique_simulants, dtype: float64

Simulated tax records with the most collisions:


,source_record_id,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,simulant_id
0,simulated_1040_26_10555570,simulated_taxes_26_10555570,808-26-5998,Justin,A,Koscielski,114,w 5th st,<NA>,<NA>,doraville,GA,31906,6554_95406
1,simulated_1040_36_18164191,simulated_taxes_36_18164191,808-26-5998,Christopher,W,Palm,114,w 5th st,<NA>,<NA>,doraville,GA,31906,6554_95394
2,simulated_1040_36_18164199,simulated_taxes_36_18164199,808-26-5998,Justin,A,Koscielski,114,w 5th st,<NA>,<NA>,doraville,GA,31906,6554_95406
3,simulated_1040_6_13639346,simulated_taxes_6_13639346,808-26-5998,Justin,A,Kosc,114,w 5th st,<NA>,<NA>,doraville,GA,31906,6554_95406
4,simulated_w2_1099_16_28733907,simulated_taxes_67_28733907,808-26-5998,David,D,Romero,114,w 5th st,<NA>,<NA>,doraville,GA,31906,6554_95389
5,simulated_w2_1099_16_28733911,simulated_taxes_67_28733911,808-26-5998,Bradley,,M So,114,w 5th st,<NA>,<NA>,doraville,GA,31906,6554_95392
6,simulated_w2_1099_16_28733915,simulated_taxes_67_28733915,808-26-5998,Elijah,M,Elizondo,114,w 5th st,<NA>,<NA>,doraville,GA,31906,6554_95393
7,simulated_w2_1099_16_28733918,simulated_taxes_67_28733918,808-26-5998,Justin,J,Flores,114,w 5th st,<NA>,<NA>,doraville,GA,31906,6554_95396
8,simulated_w2_1099_16_28733919,simulated_taxes_67_28733919,808-26-5998,Alec,A,Viera Salazar,114,w 5th st,<NA>,<NA>,doraville,GA,31906,6554_95397
9,simulated_w2_1099_16_28733932,simulated_taxes_67_28733932,808-26-5998,Justin,A,Koscielski,114,w 5th st,<NA>,<NA>,doraville,GA,31906,6554_95406


Simulated SSA records with the most collisions:


,source_record_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,simulant_id
0,simulated_ssa_numident_223_681571,808-26-5998,Justin,Austin,Koscielski,19960210,Male,creation,19960210,6554_95406


In [87]:
simulated_geobase_reference_file_ground_truth

,record_id,simulant_id,n_unique_simulants
npartitions=53,,,
,string,string,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [88]:
%xdel simulated_taxes
%xdel simulated_taxes_source_record_pairs

### Get ground truth by SSN

In [89]:
simulated_ssn_simulant_pairs = df_ops.persist(
    df_ops.concat([
        simulated_census_numident[["record_id", "ssn"]].merge(simulated_census_numident_ground_truth, on="record_id"),
        simulated_alternate_name_numident[["record_id", "ssn"]].merge(simulated_alternate_name_numident_ground_truth, on="record_id"),
        simulated_alternate_dob_numident[["record_id", "ssn"]].merge(simulated_alternate_dob_numident_ground_truth, on="record_id"),
        simulated_name_dob_reference_file[["record_id", "ssn"]].merge(simulated_name_dob_reference_file_ground_truth, on="record_id"),
        simulated_geobase_reference_file[["record_id", "ssn"]].merge(simulated_geobase_reference_file_ground_truth, on="record_id"),
    ])
        [['ssn', 'simulant_id']]
        .pipe(df_ops.drop_duplicates)
)
simulated_ssn_simulant_pairs

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    3.692000e+03
mean     6.584480e+07
std      9.184251e+07
min      3.938182e+07
25%      4.323577e+07
50%      4.428586e+07
75%      4.724390e+07
max      4.458707e+08
dtype: float64
Creating partitions of 4,862MB


,ssn,simulant_id
npartitions=104,,
,string,string
,...,...
...,...,...
,...,...
,...,...


In [90]:
# How many simulated SSNs have different simulant IDs
# contributing to them?
df_ops.compute((df_ops.groupby_agg_small_groups(simulated_ssn_simulant_pairs, by='ssn', agg_func=lambda x: x.simulant_id.nunique()) > 1).sum())

44706617

## Save results

In [91]:
files = {
    'simulated_census_numident': (simulated_census_numident, simulated_census_numident_ground_truth),
    'simulated_alternate_name_numident': (simulated_alternate_name_numident, simulated_alternate_name_numident_ground_truth),
    'simulated_alternate_dob_numident': (simulated_alternate_dob_numident, simulated_alternate_dob_numident_ground_truth),
    'simulated_geobase_reference_file': (simulated_geobase_reference_file, simulated_geobase_reference_file_ground_truth),
    'simulated_name_dob_reference_file': (simulated_name_dob_reference_file, simulated_name_dob_reference_file_ground_truth),
}

In [92]:
simulated_reference_files = [
    simulated_census_numident,
    simulated_alternate_name_numident,
    simulated_alternate_dob_numident,
    simulated_geobase_reference_file,
    simulated_name_dob_reference_file,
]
# TODO: Rename the ssn column to explicitly include itins, since this is confusing
simulated_all_ssns_itins_in_reference_files = df_ops.concat([df[["ssn"]] for df in simulated_reference_files], ignore_index=True)
simulated_ssn_to_pik = (
    simulated_all_ssns_itins_in_reference_files
        .pipe(df_ops.drop_duplicates)
        .pipe(add_unique_id_col, col_name='pik')
        [['ssn', 'pik']]
)
simulated_ssn_to_pik

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    1.753000e+03
mean     2.849045e+07
std      4.588810e+07
min      0.000000e+00
25%      0.000000e+00
50%      9.174480e+05
75%      4.522642e+07
max      2.078410e+08
dtype: float64
Creating partitions of 999MB


,ssn,pik
npartitions=106,,
,string,string
,...,...
...,...,...
,...,...
,...,...


In [93]:
simulated_pik_simulant_pairs = (
    simulated_ssn_simulant_pairs
        .merge(simulated_ssn_to_pik, on="ssn", how="inner")
        [['pik', 'simulant_id']]
)
simulated_pik_simulant_pairs

,pik,simulant_id
npartitions=106,,
,string,string
,...,...
...,...,...
,...,...
,...,...


In [94]:
%xdel simulated_ssn_simulant_pairs

In [95]:
for file_name, (file, ground_truth) in files.items():
    file = df_ops.persist(file.merge(simulated_ssn_to_pik, on='ssn', how='left'))
    assert df_ops.compute(file.pik.notnull().all())

    save_file_with_ground_truth(file_name, file, ground_truth)

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    1.336000e+03
mean     1.537621e+07
std      2.650426e+07
min      0.000000e+00
25%      0.000000e+00
50%      5.665000e+03
75%      1.150296e+07
max      1.362955e+08
dtype: float64
Creating partitions of 411MB


Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    1.336000e+03
mean     1.488268e+07
std      2.580343e+07
min      0.000000e+00
25%      0.000000e+00
50%      5.436000e+03
75%      1.026907e+07
max      1.304817e+08
dtype: float64
Creating partitions of 398MB


In [96]:
simulated_pik_simulant_pairs_path = f'{output_dir}/{data_to_use}/simulated_pik_simulant_pairs.parquet'
utils.remove_path(simulated_pik_simulant_pairs_path)
simulated_pik_simulant_pairs.to_parquet(simulated_pik_simulant_pairs_path)

In [97]:
print(datetime.datetime.now())

2024-05-22 18:11:56.874987
